In [2]:
import pickle
import os
from od.social import SocialGroup
from od.network.types import ResourceAllocatorType,BaseStationType
from od.misc.interest import InterestConfig
from od.config import BS_PRESET,SUMO_SIM_SECONDS

In [18]:
interest_config = InterestConfig(ResourceAllocatorType.NOMA_OPT,True,1.0,1)
statistic = None
print(interest_config)
with open("data/{}/{}/object.pickle".format(interest_config.rng_seed,str(interest_config)), "rb") as file:
    statistic = pickle.load(file)

res_alloc_type(NOMA_OPT) req_rsu(True) traffic_scale(1.0)


In [19]:
class GV:
    NET_STATION_CONTROLLER = [
            bs_config for bs_config in BS_PRESET.values()
             if ((bs_config["type"] == BaseStationType.UMA)  or 
            (interest_config.req_rsu and bs_config["type"] == BaseStationType.UMI))
    ]
class STATISTIC:
    def Log(data):
        print(data)

In [20]:
GV.NET_STATION_CONTROLLER

[{'color': (0, 0, 0, 255),
  'pos': (160, 100),
  'img': '/home/OD/repos/ProjectSumo/wifi.png',
  'width': 5,
  'height': 3.9,
  'type': <BaseStationType.UMA: 1>},
 {'color': (0, 0, 0, 255),
  'pos': (450, 330),
  'img': '/home/OD/repos/ProjectSumo/wifi.png',
  'width': 5,
  'height': 3.9,
  'type': <BaseStationType.UMA: 1>},
 {'color': (0, 0, 0, 255),
  'pos': (127, 217),
  'img': '/home/OD/repos/ProjectSumo/wifi.png',
  'width': 5,
  'height': 3.9,
  'type': <BaseStationType.UMI: 0>},
 {'color': (0, 0, 0, 255),
  'pos': (52, 145),
  'img': '/home/OD/repos/ProjectSumo/wifi.png',
  'width': 5,
  'height': 3.9,
  'type': <BaseStationType.UMI: 0>},
 {'color': (0, 0, 0, 255),
  'pos': (315, 395),
  'img': '/home/OD/repos/ProjectSumo/wifi.png',
  'width': 5,
  'height': 3.9,
  'type': <BaseStationType.UMI: 0>},
 {'color': (0, 0, 0, 255),
  'pos': (270, 50),
  'img': '/home/OD/repos/ProjectSumo/wifi.png',
  'width': 5,
  'height': 3.9,
  'type': <BaseStationType.UMI: 0>},
 {'color': (0, 0, 

In [21]:
print(len(statistic.sg_header[SocialGroup.RCWS].keys()))
print(len(statistic.sg_header[SocialGroup.CRASH].keys()))

20158
5591


In [22]:

for sg in SocialGroup:
    ot_count = 0 
    for header, record in statistic.sg_header[sg].items():
        if record.is_src_ot > 0:
            ot_count +=1
    print(ot_count)
    print(ot_count / len(statistic.sg_header[sg]))
            

0
0.0
0
0.0


In [23]:

class Helper:
    def __init__(self,header,record,bs_index):
        self.name = header
        self.bits = record.bits
        self.time_gen = round(record.at,4)
        self.time_serv = round(record.time_bs_serv[bs_index],4)
        self.time_len = round(self.time_serv - self.time_gen,4)
    def __str__(self):
        return "{}[ at:{} ,to:{}, dur:{}, size:{}]".format(self.name,self.time_gen,self.time_serv,self.time_len,self.bits)
    def __repr__(self):
        return "{}[ at:{} ,to:{}, dur:{}, size:{}]".format(self.name,self.time_gen,self.time_serv,self.time_len,self.bits)


In [24]:
target_header = "flow_0.21-110"
print("C2G?:{}".format(target_header in statistic.sg_header[SocialGroup.RCWS] and target_header in statistic.sg_header[SocialGroup.CRASH]))

C2G?:False


In [25]:
statistic.sg_header[SocialGroup.RCWS][target_header].at

186.257

In [27]:
statistic.sg_header[SocialGroup.RCWS][target_header].time_bs_serv

[-1, 186.3, -1, -1, -1, -1, -1]

In [29]:
bs_serv_list = []
for header, record in statistic.sg_header[SocialGroup.RCWS].items():
    if(record.time_bs_serv[1] > -1):
        bs_serv_list.append(Helper(header,record,1))
bs_serv_list.sort(key=lambda x: (x.time_gen*1000)*10000000 + (x.time_len*1000) )

In [30]:
bs_serv_list_crit = []
for header, record in statistic.sg_header[SocialGroup.CRASH].items():
    if(record.time_bs_serv[1] > -1):
        bs_serv_list_crit.append(Helper(header,record,1))
bs_serv_list_crit.sort(key=lambda x: (x.time_gen*1000)*10000000 + (x.time_len*1000))

In [31]:
for helper in bs_serv_list_crit:
    print(str(helper))

flow_8.2-82[ at:186.014 ,to:186.015, dur:0.001, size:3840]
flow_1.4-82[ at:186.018 ,to:186.019, dur:0.001, size:7992]
flow_2.62-83[ at:186.018 ,to:186.019, dur:0.001, size:7760]
flow_2.61-83[ at:186.019 ,to:186.02, dur:0.001, size:6552]
flow_2.63-82[ at:186.019 ,to:186.02, dur:0.001, size:7504]
flow_0.26-83[ at:186.02 ,to:186.021, dur:0.001, size:7416]
flow_2.60-83[ at:186.021 ,to:186.022, dur:0.001, size:8736]
flow_0.25-84[ at:186.025 ,to:186.026, dur:0.001, size:6712]
flow_8.2-86[ at:186.047 ,to:186.048, dur:0.001, size:6792]
flow_1.4-86[ at:186.05 ,to:186.051, dur:0.001, size:2712]
flow_0.26-87[ at:186.052 ,to:186.053, dur:0.001, size:7824]
flow_2.62-86[ at:186.052 ,to:186.053, dur:0.001, size:4976]
flow_2.61-87[ at:186.054 ,to:186.055, dur:0.001, size:7600]
flow_2.63-86[ at:186.054 ,to:186.055, dur:0.001, size:3616]
flow_2.60-87[ at:186.055 ,to:186.056, dur:0.001, size:7184]
flow_0.25-87[ at:186.059 ,to:186.06, dur:0.001, size:5752]
flow_8.2-90[ at:186.081 ,to:186.082, dur:0.001, s

In [32]:
for helper in bs_serv_list:
    print(str(helper))

flow_0.19-81[ at:186.0 ,to:186.001, dur:0.001, size:3688]
flow_0.22-81[ at:186.0 ,to:186.001, dur:0.001, size:10520]
flow_0.25-81[ at:186.0 ,to:186.001, dur:0.001, size:14448]
flow_0.26-81[ at:186.001 ,to:186.002, dur:0.001, size:10440]
flow_2.56-81[ at:186.001 ,to:186.002, dur:0.001, size:11632]
flow_2.60-81[ at:186.001 ,to:186.002, dur:0.001, size:6488]
flow_3.1-81[ at:186.001 ,to:186.002, dur:0.001, size:6368]
flow_1.3-81[ at:186.002 ,to:186.003, dur:0.001, size:8592]
flow_2.48-81[ at:186.002 ,to:186.003, dur:0.001, size:2056]
flow_0.11-81[ at:186.003 ,to:186.004, dur:0.001, size:4472]
flow_0.14-81[ at:186.003 ,to:186.004, dur:0.001, size:3960]
flow_0.17-81[ at:186.003 ,to:186.004, dur:0.001, size:3304]
flow_2.46-81[ at:186.003 ,to:186.004, dur:0.001, size:3984]
flow_2.50-80[ at:186.003 ,to:186.004, dur:0.001, size:2424]
flow_2.51-81[ at:186.003 ,to:186.004, dur:0.001, size:1928]
flow_2.53-81[ at:186.003 ,to:186.004, dur:0.001, size:10384]
flow_2.58-81[ at:186.003 ,to:186.004, dur:0

In [33]:
time_gap = {}
for helper in bs_serv_list_crit:
    at = str(helper.time_gen)
    to = str(helper.time_serv)
    if(to not in  time_gap):
        time_gap[to] = {"need": 0, "give": 0}
    if(at not in time_gap):
        time_gap[at] = {"need": 0, "give": 0}
    time_gap[to]["give"]+=helper.bits
    time_gap[at]["need"]+=helper.bits
keys = list(time_gap.keys())
keys.sort()
for key in keys:
    print("{}:{}".format(key,str(time_gap[key])))

186.014:{'need': 3840, 'give': 0}
186.015:{'need': 0, 'give': 3840}
186.018:{'need': 15752, 'give': 0}
186.019:{'need': 14056, 'give': 15752}
186.02:{'need': 7416, 'give': 14056}
186.021:{'need': 8736, 'give': 7416}
186.022:{'need': 0, 'give': 8736}
186.025:{'need': 6712, 'give': 0}
186.026:{'need': 0, 'give': 6712}
186.047:{'need': 6792, 'give': 0}
186.048:{'need': 0, 'give': 6792}
186.05:{'need': 2712, 'give': 0}
186.051:{'need': 0, 'give': 2712}
186.052:{'need': 12800, 'give': 0}
186.053:{'need': 0, 'give': 12800}
186.054:{'need': 11216, 'give': 0}
186.055:{'need': 7184, 'give': 11216}
186.056:{'need': 0, 'give': 7184}
186.059:{'need': 5752, 'give': 0}
186.06:{'need': 0, 'give': 5752}
186.081:{'need': 3952, 'give': 0}
186.082:{'need': 0, 'give': 3952}
186.084:{'need': 8056, 'give': 0}
186.085:{'need': 5928, 'give': 8056}
186.086:{'need': 4336, 'give': 5928}
186.087:{'need': 0, 'give': 4336}
186.088:{'need': 4328, 'give': 0}
186.089:{'need': 0, 'give': 4328}
186.09:{'need': 4152, 'gi

In [35]:
time_gap = {}
for helper in bs_serv_list:
    at = str(helper.time_gen)
    to = str(helper.time_serv)
    if(to not in  time_gap):
        time_gap[to] = {"need": 0, "give": 0}
    if(at not in time_gap):
        time_gap[at] = {"need": 0, "give": 0}
    time_gap[to]["give"]+=helper.bits
    time_gap[at]["need"]+=helper.bits
keys = list(time_gap.keys())
keys.sort()
for key in keys:
    print("{}:{}".format(key,str(time_gap[key])))

186.0:{'need': 28656, 'give': 0}
186.001:{'need': 34928, 'give': 28656}
186.002:{'need': 10648, 'give': 34928}
186.003:{'need': 38736, 'give': 10648}
186.004:{'need': 32192, 'give': 38736}
186.005:{'need': 78840, 'give': 32192}
186.006:{'need': 16856, 'give': 47808}
186.007:{'need': 50344, 'give': 47888}
186.008:{'need': 15760, 'give': 50344}
186.009:{'need': 1040, 'give': 15760}
186.01:{'need': 6784, 'give': 1040}
186.011:{'need': 9872, 'give': 6784}
186.012:{'need': 45488, 'give': 9872}
186.013:{'need': 21992, 'give': 45488}
186.014:{'need': 52192, 'give': 21992}
186.015:{'need': 60320, 'give': 52192}
186.016:{'need': 60248, 'give': 49200}
186.017:{'need': 68008, 'give': 58920}
186.018:{'need': 134424, 'give': 53360}
186.019:{'need': 96400, 'give': 51216}
186.02:{'need': 71904, 'give': 43864}
186.021:{'need': 57488, 'give': 51472}
186.022:{'need': 22992, 'give': 50112}
186.023:{'need': 51960, 'give': 46504}
186.024:{'need': 40352, 'give': 67304}
186.025:{'need': 62176, 'give': 56544}

In [23]:
list(time_gap.keys()).sort()